In [1]:
import matplotlib.pyplot as plt
import numpy as np


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2

In [4]:
 pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [5]:
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [6]:
import urllib.request
import os
import tarfile

In [ ]:
url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
filename = "flower_photos.tgz"
urllib.request.urlretrieve(url,filename)

# Extract the dataset
with tarfile.open(filename, "r:gz") as tar:
    tar.extractall()

# Check if the extraction is correct
extracted_folder = 'flower_photos'
assert os.path.exists(extracted_folder), "Extraction failed or folder path is incorrect"

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam


In [9]:
#define path
base_dir = 'flower_photos'
categories = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [10]:
#create directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)

In [11]:
import shutil
import random

split_ratio = 0.8

for category in categories:
    category_path = os.path.join(base_dir, category)
    images = os.listdir(category_path)
    random.shuffle(images)
    split_index = int(len(images) * split_ratio)
    
    train_images = images[:split_index]
    validation_images = images[split_index:]
    
    for image in train_images:
        shutil.move(os.path.join(category_path, image), os.path.join(train_dir, category,image))
        
    for image in validation_images:
        shutil.move(os.path.join(category_path, image), os.path.join(validation_dir, category,image))

In [ ]:
#DATA AUGMENTATION

In [12]:
IMAGE_SIZE = 150
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [13]:
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 2934 images belonging to 5 classes.
Found 736 images belonging to 5 classes.


In [14]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights= 'imagenet', include_top=False,input_shape= (IMAGE_SIZE, IMAGE_SIZE, 3))

model = Sequential([
    conv_base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

conv_base.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10


C:\Users\gayatri\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


36/92 ━━━━━━━━━━━━━━━━━━━━ 3:42 4s/step - accuracy: 0.3368 - loss: 2.1150

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc:.4f}')

# Plot training & validation accuracy and loss
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

plot_history(history)
plot_history(history_fine)



In [ ]:
model.save('flowers_classifier.h5')